In [31]:
import pandas as pd
import joblib
import pickle


In [1]:
%%bigquery df
select * from `felipe-sandbox.demos.financial_fraud`

Downloading: 100%|██████████| 6362620/6362620 [00:07<00:00, 799797.31rows/s] 


In [2]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,241,DEBIT,4.28,C1559447787,11799.0,11794.72,C459241388,974769.79,974774.07,0,0
1,254,DEBIT,3897.94,C432144892,15119.0,11221.06,C333806893,885113.68,889011.62,0,0
2,254,DEBIT,424.74,C1098271234,28824.0,28399.26,C1213279510,214316.63,214741.37,0,0
3,255,DEBIT,3888.71,C1619994413,44212.0,40323.29,C2001746908,137483.79,141372.49,0,0
4,255,DEBIT,4558.93,C1855969717,0.0,0.00,C1915784536,3147902.59,3152461.51,0,0


In [13]:
from sklearn import tree
X = df[["type","amount","oldbalanceOrg","newbalanceOrig", "oldbalanceDest","newbalanceDest"]]
X["type"] = pd.Categorical(X["type"])
X["type"] = X["type"].cat.codes
y = df["isFraud"]
X.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,2,4.28,11799.0,11794.72,974769.79,974774.07
1,2,3897.94,15119.0,11221.06,885113.68,889011.62
2,2,424.74,28824.0,28399.26,214316.63,214741.37
3,2,3888.71,44212.0,40323.29,137483.79,141372.49
4,2,4558.93,0.0,0.00,3147902.59,3152461.51


In [14]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, y)

In [35]:
joblib.dump(clf, "model.joblib")
pickle.dumps(clf "model.pkl")

SyntaxError: invalid syntax (1696891005.py, line 2)

In [38]:
!gsutil cp model.joblib gs://felipe-sandbox-demo/python_custom_code/

Copying file://model.joblib [Content-Type=application/octet-stream]...
/ [1 files][183.3 KiB/183.3 KiB]                                                
Operation completed over 1 objects/183.3 KiB.                                    


In [39]:
! gcloud ai models upload \
  --region=us-central1 \
  --display-name=financial_fraud_custom_model \
  --container-image-uri=us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest \
  --artifact-uri=gs://felipe-sandbox-demo/python_custom_code

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [5863021579021058048]...done.                            


To take a quick anonymous survey, run:
  $ gcloud survey

